In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report, hamming_loss, f1_score, make_scorer, accuracy_score
import joblib
import re
from skmultilearn.problem_transform import ClassifierChain



# --- 1. Tiền xử lý với loại bỏ Stop Words ---
VIETNAMESE_STOP_WORDS = set([
    "và", "là", "có", "được", "của", "cho", "thì", "mà", "ở", "để", "đã", "bị", "ra", "vào",
    "khi", "thấy", "tại", "bởi", "vì", "sao", "nên", "nếu", "hay", "còn", "lại", "đến", "đi", "tới",
    "luôn", "rất", "quá", "cũng", "vẫn", "chứ", "như", "nào", "gì", "ai", "đâu", "mấy",
    "một", "hai", "ba", "bốn", "năm", "sáu", "bảy", "tám", "chín", "mười",
    "anh", "em", "chị", "tôi", "bạn", "mình",
    "rằng", "thực sự", "hoàn toàn", "chắc chắn",
    "ạ", "nhé", "nha", "ơi", "đấy", "vậy", "ấy",
    "lần", "việc", "cách", "điều",
    "rồi", "hết", "ngay", "luôn", "liền",
    "xin", "hỏi", "cho hỏi", "vui lòng",
    "nó", "chúng", "họ", "kia", "đó",
    "cái", "chiếc", "trong", "ngoài", "trên", "dưới", "sau", "trước",
    "ngày", "tháng"
])

def preprocess_text_with_stopwords_removal(text):
    if not isinstance(text, str): return ""
    text = text.lower()
    text = re.sub(r'[^\w\s\u00C0-\u1EF9]', ' ', text)
    text = re.sub(r'\d+', ' <number> ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    words = text.split()
    words = [word for word in words if word not in VIETNAMESE_STOP_WORDS and len(word) > 1]
    return " ".join(words)

# --- 2. Tải Dữ liệu ---
try:
    df_original = pd.read_csv('laptop_qa_full.csv')
    df = df_original.copy()
except FileNotFoundError:
    print("Lỗi: File 'laptop_qa_full.csv' không tìm thấy.")
    exit()

# --- 3. EDA ---
print("\n--- Phân tích Dữ liệu Thăm dò (EDA) ---")
label_names_eda = [col for col in df.columns if col.startswith('intent_')]
label_counts = df[label_names_eda].sum().sort_values(ascending=False)
print("\nTần suất nhãn:\n", label_counts)

# --- 4. Tiền xử lý ---
print("\n--- Áp dụng tiền xử lý văn bản (Có loại bỏ Stop Words) ---")
df['processed_question'] = df['question'].astype(str).apply(preprocess_text_with_stopwords_removal)

# --- 5. Tách Dữ liệu ---
X_processed_sw = df['processed_question']
y = df.drop(columns=['question', 'processed_question'])
# Lấy danh sách các cột nhãn tự động
label_names = [col for col in y.columns if 'intent_' in col]
y = y[label_names] # Đảm bảo y chỉ chứa các cột nhãn
y_sum_labels = y.sum(axis=1)
stratify_param = y_sum_labels if len(np.unique(y_sum_labels)) >= 2 else None
X_trainval_sw, X_test_sw, y_trainval, y_test = train_test_split(
    X_processed_sw, y, test_size=0.25, random_state=42, stratify=stratify_param
)
y_trainval_sum_labels = y_trainval.sum(axis=1)
stratify_trainval_param = y_trainval_sum_labels if len(np.unique(y_trainval_sum_labels)) >= 2 else None
X_train_sw, X_dev_sw, y_train, y_dev = train_test_split(
    X_trainval_sw, y_trainval, test_size=0.2, random_state=42, stratify=stratify_trainval_param
)
print("\n--- Kích thước các tập dữ liệu ---")
print(f"Train size: {len(X_train_sw)}")
print(f"Dev size  : {len(X_dev_sw)}")
print(f"Test size : {len(X_test_sw)}")


# --- 6. GridSearchCV để tìm siêu tham số tốt nhất ---
print("\n\n--- GridSearchCV (Thử nghiệm nhiều chiến lược) ---")
pipeline_gs = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', None)
])

parameters = [
    {'tfidf__ngram_range': [(1, 2), (1, 3)], 'tfidf__max_df': [0.9, 1.0], 'tfidf__min_df': [1, 2], 'clf': [OneVsRestClassifier(estimator=LogisticRegression(solver='liblinear', class_weight='balanced', random_state=42), n_jobs=-1)], 'clf__estimator__C': [1, 10]},
    {'tfidf__ngram_range': [(1, 2), (1, 3)], 'tfidf__max_df': [0.9, 1.0], 'tfidf__min_df': [1, 2], 'clf': [OneVsRestClassifier(estimator=LinearSVC(class_weight='balanced', dual='auto', random_state=42, max_iter=3000), n_jobs=-1)], 'clf__estimator__C': [0.1, 1]},
    {'tfidf__ngram_range': [(1, 2), (1, 3)], 'tfidf__max_df': [0.9, 1.0], 'tfidf__min_df': [1, 2], 'clf': [OneVsRestClassifier(estimator=MultinomialNB(), n_jobs=-1)], 'clf__estimator__alpha': [0.1, 0.5]},
    {'tfidf__ngram_range': [(1, 2), (1, 3)], 'tfidf__max_df': [0.9, 1.0], 'tfidf__min_df': [1, 2], 'clf': [ClassifierChain(classifier=LogisticRegression(solver='liblinear', class_weight='balanced', random_state=42), require_dense=[False, True])], 'clf__classifier__C': [1, 10]}]

print("\n--- Định nghĩa bộ độ đo tập trung cho GridSearchCV ---")
scoring = {
    'f1_macro': make_scorer(f1_score, average='macro', zero_division=0),
    'exact_match': make_scorer(accuracy_score),
    'hamming': make_scorer(hamming_loss, greater_is_better=False),
    'f1_micro': make_scorer(f1_score, average='micro', zero_division=0) # Vẫn giữ để refit
}

grid_search = GridSearchCV(pipeline_gs, parameters, cv=3,
                           scoring=scoring,
                           refit='f1_micro',
                           verbose=2, n_jobs=-1)


print("Bắt đầu GridSearchCV (với dữ liệu CÓ loại bỏ stop words)...")
grid_search.fit(X_train_sw, y_train.values)

print("\n--- Kết quả GridSearchCV ---")
print("Chiến lược và siêu tham số tốt nhất (tối ưu theo F1-micro):")
print(grid_search.best_params_)
print(f"\nĐiểm F1-micro tốt nhất trên cross-validation: {grid_search.best_score_:.4f}")

print("\n\n--- Phân tích Hiệu suất các Thuật toán trên Cross-Validation ---")
results_df = pd.DataFrame(grid_search.cv_results_)
def get_classifier_strategy(param_dict):
    clf = param_dict.get('clf')
    if clf is not None:
        if isinstance(clf, OneVsRestClassifier):
            estimator_name = type(clf.estimator).__name__
            return f"{estimator_name} (OneVsRest)"
        elif isinstance(clf, ClassifierChain):
            classifier_name = type(clf.classifier).__name__
            return f"{classifier_name} (ClassifierChain)"
    return "Unknown"
results_df['strategy'] = results_df['params'].apply(get_classifier_strategy)
best_scores_per_strategy = results_df.loc[results_df.groupby('strategy')['mean_test_f1_macro'].idxmax()]
analysis_columns = {
    'strategy': 'Chiến lược',
    'mean_test_f1_macro': 'F1 Macro (CV)',
    'mean_test_exact_match': 'Khớp Chính xác (CV)',
    'mean_test_hamming': 'Hamming Loss (CV)'
}
final_analysis_df = best_scores_per_strategy[list(analysis_columns.keys())].rename(columns=analysis_columns)
if 'Hamming Loss (CV)' in final_analysis_df.columns:
    final_analysis_df['Hamming Loss (CV)'] = final_analysis_df['Hamming Loss (CV)'].abs()
final_analysis_df = final_analysis_df.sort_values(by='F1 Macro (CV)', ascending=False)
print("\nBảng so sánh hiệu suất tốt nhất của mỗi thuật toán/chiến lược (sắp xếp theo F1 Macro):")
print("Giải thích các cột:")
print(" - F1 Macro: Hiệu suất trung bình trên các nhãn, coi mỗi nhãn quan trọng như nhau. Càng cao càng tốt.")
print(" - Khớp Chính xác: Tỷ lệ câu hỏi được phân loại đúng hoàn toàn (nghiêm ngặt). Càng cao càng tốt.")
print(" - Hamming Loss: Tỷ lệ nhãn bị dự đoán sai. Càng thấp càng tốt.")
print("-" * 80)
print(final_analysis_df.to_string(index=False, float_format="%.4f"))



# --- 7. Đánh giá Mô hình Tốt nhất trên Tập Dev ---
best_model = grid_search.best_estimator_
y_dev_pred = best_model.predict(X_dev_sw)
print("\n\n--- Đánh giá Mô hình Tốt nhất trên Tập Dev ---")
print(classification_report(y_dev, y_dev_pred, target_names=label_names, zero_division=0))
# Báo cáo bộ độ đo đã tinh chỉnh
print("\n--- Tổng hợp các độ đo trên tập Dev ---")
print(f"F1-score (macro): {f1_score(y_dev, y_dev_pred, average='macro', zero_division=0):.4f}  (Hiệu suất trung bình trên các nhãn)")
print(f"Exact Match Ratio: {accuracy_score(y_dev, y_dev_pred):.4f} (Tỷ lệ dự đoán đúng hoàn toàn)")
print(f"Hamming Loss: {hamming_loss(y_dev, y_dev_pred):.4f} (Tỷ lệ nhãn bị lỗi, càng thấp càng tốt)")


# --- 8. Phân tích lỗi  ---
print("\n\n--- Phân tích lỗi chi tiết và thống kê trên tập Dev ---")

# Các hằng số để dễ dàng tùy chỉnh
NUM_EXAMPLES_TO_SHOW = 5
TOP_N_ERRORS = 10

# Chuẩn bị dữ liệu để phân tích
errors_list = []
dev_indices = X_dev_sw.index
y_dev_values = y_dev.values

# Dùng dictionary để đếm lỗi
fp_counts = {} # False Positives: dự đoán nhãn A nhưng thực tế không có
fn_counts = {} # False Negatives: thực tế có nhãn A nhưng không dự đoán ra

for i in range(len(X_dev_sw)):
    actual_labels_arr = y_dev_values[i]
    
    # Hỗ trợ cả sparse matrix (từ ClassifierChain) và dense array
    if hasattr(y_dev_pred, 'toarray'):
        predicted_labels_arr = y_dev_pred[i].toarray()[0]
    else:
        predicted_labels_arr = y_dev_pred[i]
        
    # Chỉ xử lý những trường hợp có lỗi
    if not np.array_equal(actual_labels_arr, predicted_labels_arr):
        original_question_index = dev_indices[i]
        
        # Chuyển từ mảng 0/1 sang tập hợp (set) các tên nhãn để dễ so sánh
        actual_set = {label_names[j] for j, val in enumerate(actual_labels_arr) if val == 1}
        predicted_set = {label_names[j] for j, val in enumerate(predicted_labels_arr) if val == 1}
        
        # Sử dụng phép toán trên tập hợp để tìm lỗi
        false_positives = predicted_set - actual_set
        false_negatives = actual_set - predicted_set
        
        # Lưu thông tin lỗi vào danh sách
        errors_list.append({
            "original_question": df_original.loc[original_question_index, 'question'],
            "processed_question": X_dev_sw.iloc[i],
            "actual": actual_set if actual_set else {"Không có"},
            "predicted": predicted_set if predicted_set else {"Không có"},
            "false_positives": false_positives,
            "false_negatives": false_negatives
        })
        
        # Cập nhật bộ đếm thống kê
        for label in false_positives:
            fp_counts[label] = fp_counts.get(label, 0) + 1
        for label in false_negatives:
            fn_counts[label] = fn_counts.get(label, 0) + 1

# --- Báo cáo Kết quả Phân tích Lỗi ---
if not errors_list:
    print("\n✅ Không tìm thấy lỗi nào trên tập Dev. Mô hình hoạt động hoàn hảo!")
else:
    total_errors = len(errors_list)
    total_samples = len(X_dev_sw)
    print(f"\nTổng quan: Tìm thấy {total_errors}/{total_samples} mẫu bị lỗi ({total_errors/total_samples:.2%}).")

    # 1. Thống kê các nhãn bị lỗi nhiều nhất
    print("\n--- Thống kê Lỗi ---")
    
    # Sắp xếp và in ra các nhãn bị thêm sai nhiều nhất (False Positives)
    if fp_counts:
        sorted_fp = sorted(fp_counts.items(), key=lambda item: item[1], reverse=True)
        print(f"\nTop {TOP_N_ERRORS} nhãn bị THÊM SAI nhiều nhất (False Positives):")
        for label, count in sorted_fp[:TOP_N_ERRORS]:
            print(f"  - {label}: {count} lần")
            
    # Sắp xếp và in ra các nhãn bị bỏ sót nhiều nhất (False Negatives)
    if fn_counts:
        sorted_fn = sorted(fn_counts.items(), key=lambda item: item[1], reverse=True)
        print(f"\nTop {TOP_N_ERRORS} nhãn bị BỎ SÓT nhiều nhất (False Negatives):")
        for label, count in sorted_fn[:TOP_N_ERRORS]:
            print(f"  - {label}: {count} lần")

    # 2. In ra ví dụ lỗi chi tiết
    print(f"\n\n--- Phân tích {min(NUM_EXAMPLES_TO_SHOW, len(errors_list))} ví dụ lỗi chi tiết ---")
    for i, error in enumerate(errors_list[:NUM_EXAMPLES_TO_SHOW]):
        print(f"\n----------- Lỗi #{i+1} -----------")
        print(f"Câu hỏi gốc: {error['original_question']}")
        # print(f" Đã xử lý  : {error['processed_question']}")
        print(f"Nhãn thực tế  : {sorted(list(error['actual']))}")
        print(f"Nhãn dự đoán : {sorted(list(error['predicted']))}")
        if error['false_positives']:
            print(f"Bị thêm sai (FP) : {sorted(list(error['false_positives']))}")
        if error['false_negatives']:
            print(f"Bị bỏ sót (FN)  : {sorted(list(error['false_negatives']))}")


# --- 9. Lưu trữ Mô hình Tốt nhất ---
model_filename = 'best_intent_classifier_stopwords_removed.joblib'
joblib.dump(best_model, model_filename)
print(f"\n\n--- Mô hình tốt nhất đã được lưu vào: {model_filename} ---")


# --- 10. Đánh giá Cuối cùng trên Tập Test ---
print("\n\n--- Đánh giá CUỐI CÙNG của Mô hình Tốt nhất trên Tập Test ---")
y_test_pred = best_model.predict(X_test_sw)
print("\nFINAL Classification Report (Test set - Best GS Model):")
print(classification_report(y_test, y_test_pred, target_names=label_names, zero_division=0))

print("\n--- Tổng hợp các độ đo trên tập Test ---")
print(f"FINAL F1-score (macro): {f1_score(y_test, y_test_pred, average='macro', zero_division=0):.4f}")
print(f"FINAL Exact Match Ratio: {accuracy_score(y_test, y_test_pred):.4f}")
print(f"FINAL Hamming Loss: {hamming_loss(y_test, y_test_pred):.4f}")

print("\n--- HOÀN THÀNH QUY TRÌNH ---")


--- Phân tích Dữ liệu Thăm dò (EDA) ---

Tần suất nhãn:
 intent_tech_detail         4504
intent_recommend_usage     3301
intent_recommend_budget    2395
dtype: int64

--- Áp dụng tiền xử lý văn bản (Có loại bỏ Stop Words) ---

--- Kích thước các tập dữ liệu ---
Train size: 4158
Dev size  : 1040
Test size : 1733


--- GridSearchCV (Thử nghiệm nhiều chiến lược) ---

--- Định nghĩa bộ độ đo tập trung cho GridSearchCV ---
Bắt đầu GridSearchCV (với dữ liệu CÓ loại bỏ stop words)...
Fitting 3 folds for each of 64 candidates, totalling 192 fits

--- Kết quả GridSearchCV ---
Chiến lược và siêu tham số tốt nhất (tối ưu theo F1-micro):
{'clf': OneVsRestClassifier(estimator=LinearSVC(class_weight='balanced', max_iter=3000,
                                        random_state=42),
                    n_jobs=-1), 'clf__estimator__C': 1, 'tfidf__max_df': 0.9, 'tfidf__min_df': 1, 'tfidf__ngram_range': (1, 2)}

Điểm F1-micro tốt nhất trên cross-validation: 0.9605


--- Phân tích Hiệu suất các Thuật t